In [ ]:
import pandas as pd
from pathlib import Path
from costumDataset import *
data_path = Path(r'..\Data')
list(data_path.glob('*'))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
csv_path = data_path/'aug_data_labels.csv'
csv_df = pd.read_csv(csv_path)
csv_df.drop_duplicates(subset=['Name'], inplace=True)
csv_df.to_csv(csv_path, index=False)

In [ ]:
#Thanks to: https://stackoverflow.com/questions/72552647/pytorch-fasterrcnn-gives-no-output
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights, FasterRCNN_ResNet50_FPN_V2_Weights
import torchvision

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

In [ ]:
import pandas as pd
from pathlib import Path
from costumDataset import *
data_path = Path(r'..\Data')
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights, FasterRCNN_ResNet50_FPN_V2_Weights
import torchvision

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
df = pd.read_csv(data_path/'aug_data_labels.csv')
df = df[df['label'] == 1]

img_dir = str(data_path/'fixed_data') + '\\'
trans = T.ToTensor()

train_ds = RipCurrentDataset(df, img_dir, trans)

train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)


for images, targets in train_dl:

    
    boxes = targets['box']
    labels = targets['labels']
    new_targets = [
        {'boxes': box, 'labels': label}
        for box, label in zip(boxes, labels)
        
    ]
    
    
    output = model(images, new_targets)
    model.eval()
    
    break



In [ ]:


predictions = model(images)


In [ ]:
predictions

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)


# For training


images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 11, 4)
boxes[:, :, 2:4] = boxes[:, :, 0:2] + boxes[:, :, 2:4]
labels = torch.randint(1, 91, (4, 11))
images = list(image for image in images)
targets = []
for i in range(len(images)):
    d = {}
    d['boxes'] = boxes[i]
    d['labels'] = labels[i]
    targets.append(d)
    
    
output = model(images, targets)


# For inference
model.eval()


x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]

predictions = model(x)



In [ ]:
d

In [ ]:
d1

In [ ]:
df = pd.read_csv(data_path/'aug_data_labels.csv')
df = df[df['label'] == 1]

img_dir = str(data_path/'fixed_data') + '\\'
trans = T.ToTensor()

train_ds = RipCurrentDataset(df, img_dir, trans)

train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)

a = next(iter(train_dl))

In [ ]:
import pandas as pd

from costumDataset import *
from resize_and_augmentation import *
from pathlib import Path

data_path = Path(r'..\Data')
S = np.array((300, 300))
data_labels_path = str(data_path/'data_labels.csv')
im_path = str(data_path/'with_rips') + '\\'
target_path = str(data_path/'fixed_data') + '\\'
fix_size_and_aug(data_labels_path, S, im_path, target_path)